In [1]:
import requests
import time
import pandas as pd
import numpy as np
import sqlalchemy

In [2]:
# client-identifier ist Voraussetzung für die Nutzung der Daten
client_header = "testuser-exercise"

headers = {
    "Client-Identifier": client_header
}

# BASE URL
URL = "https://gbfs.urbansharing.com/oslobysykkel.no/"

# endpoints
# Auto-discovery file that links to all of the other files published by the system.
auto = "gbfs.json"

# Machine readable basic information about Oslo City Bike
system = "system_information.json"

# vehicly types available
vehicles = "vehicle_types.json"

# Information about all stations, including geolocation, addresses and descriptions
stations = "station_information.json"

# Bike and dock availability
availability = "station_status.json"

# pricing plans
pricing = "system_pricing_plans.json"




In [3]:
# Überblick verschaffen
response = requests.get(URL + auto, headers = headers)


if response.status_code == 200:
    data = response.json()
    print("Systemname:", data["data"])

else: 
    print("Fehler beim abrufen der Daten:", response.status_code)

Systemname: {'nb': {'feeds': [{'name': 'system_information', 'url': 'https://gbfs.urbansharing.com/oslobysykkel.no/system_information.json'}, {'name': 'vehicle_types', 'url': 'https://gbfs.urbansharing.com/oslobysykkel.no/vehicle_types.json'}, {'name': 'system_pricing_plans', 'url': 'https://gbfs.urbansharing.com/oslobysykkel.no/system_pricing_plans.json'}, {'name': 'station_information', 'url': 'https://gbfs.urbansharing.com/oslobysykkel.no/station_information.json'}, {'name': 'station_status', 'url': 'https://gbfs.urbansharing.com/oslobysykkel.no/station_status.json'}]}}


In [4]:
system_info = requests.get(URL + system, headers = headers).json()
vehicle_types = requests.get(URL + vehicles, headers = headers).json()
station_status = requests.get(URL + availability, headers = headers).json()
station_info = requests.get(URL + stations, headers = headers).json()
# pricing_plans = requests.get(URL + pricing, headers = headers).json()


df_system_info = pd.DataFrame(system_info["data"]) # wird wahrscheinlich nicht gebraucht
df_veh_types = pd.DataFrame(vehicle_types["data"]["vehicle_types"])
df_station_status = pd.DataFrame(station_status["data"]["stations"])
df_stations = pd.DataFrame(station_info["data"]["stations"])
# df_pricing = pd.DataFrame(pricing_plans["data"]["plans"]) # in der Doku nur 2 Preisklassen für bike und ebike aber auf der Website gibt es fünf Tarife/Abos


In [5]:
df_system_info.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2 entries, android to ios
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   system_id     2 non-null      object
 1   language      2 non-null      object
 2   name          2 non-null      object
 3   operator      2 non-null      object
 4   timezone      2 non-null      object
 5   phone_number  2 non-null      object
 6   email         2 non-null      object
 7   rental_apps   2 non-null      object
dtypes: object(8)
memory usage: 144.0+ bytes


In [6]:
df_veh_types.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   vehicle_type_id          2 non-null      object 
 1   form_factor              2 non-null      object 
 2   default_pricing_plan_id  2 non-null      object 
 3   propulsion_type          2 non-null      object 
 4   name                     2 non-null      object 
 5   max_range_meters         1 non-null      float64
dtypes: float64(1), object(5)
memory usage: 228.0+ bytes


In [7]:
df_veh_types = df_veh_types.drop(columns = ["default_pricing_plan_id", "form_factor"], axis = 1)
df_veh_types

,vehicle_type_id,propulsion_type,name,max_range_meters
0,bike,human,Traditional bike,NaN
1,ebike,electric_assist,Electric bike,7000.0


In [8]:
df_station_status.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270 entries, 0 to 269
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   station_id               270 non-null    object
 1   is_installed             270 non-null    bool  
 2   is_renting               270 non-null    bool  
 3   is_returning             270 non-null    bool  
 4   last_reported            270 non-null    int64 
 5   num_vehicles_available   270 non-null    int64 
 6   num_bikes_available      270 non-null    int64 
 7   num_docks_available      270 non-null    int64 
 8   vehicle_types_available  270 non-null    object
dtypes: bool(3), int64(4), object(2)
memory usage: 13.6+ KB


In [9]:
df_stations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270 entries, 0 to 269
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   station_id          270 non-null    object 
 1   name                270 non-null    object 
 2   address             270 non-null    object 
 3   cross_street        270 non-null    object 
 4   lat                 270 non-null    float64
 5   lon                 270 non-null    float64
 6   is_virtual_station  270 non-null    bool   
 7   capacity            270 non-null    int64  
 8   station_area        270 non-null    object 
 9   rental_uris         270 non-null    object 
dtypes: bool(1), float64(2), int64(1), object(6)
memory usage: 19.4+ KB


In [12]:
df_stations.head()


,station_id,name,address,cross_street,lat,lon,is_virtual_station,capacity,station_area,rental_uris
0,5431,NRK,"Suhms gate 20A, 0362 Oslo",ved Suhms gate,59.933350,10.719603,False,21,"{'type': 'MultiPolygon', 'coordinates': [[[[10...","{'android': 'oslobysykkel://stations/5431', 'i..."
1,5424,Makrellbekken T,Arnebråtveien 1,"ved plattform retning Røa, i Arnebråtveien",59.941903,10.674101,False,18,"{'type': 'MultiPolygon', 'coordinates': [[[[10...","{'android': 'oslobysykkel://stations/5424', 'i..."
2,5220,Blindern vgs,Sognsveien,under sykkelbroen,59.949909,10.730185,False,20,"{'type': 'MultiPolygon', 'coordinates': [[[[10...","{'android': 'oslobysykkel://stations/5220', 'i..."
3,5217,Egertorget,Øvre Slottsgate 19,ved Max burger,59.912674,10.742025,False,30,"{'type': 'MultiPolygon', 'coordinates': [[[[10...","{'android': 'oslobysykkel://stations/5217', 'i..."
4,5113,Leirfallsgata,Markveien 58,ved Markveien,59.919115,10.757831,False,15,"{'type': 'MultiPolygon', 'coordinates': [[[[10...","{'android': 'oslobysykkel://stations/5113', 'i..."


In [17]:
df_stations = df_stations.drop(columns = ["station_area", "rental_uris"], axis = 1)


In [19]:
df_stations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270 entries, 0 to 269
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   station_id          270 non-null    object 
 1   name                270 non-null    object 
 2   address             270 non-null    object 
 3   cross_street        270 non-null    object 
 4   lat                 270 non-null    float64
 5   lon                 270 non-null    float64
 6   is_virtual_station  270 non-null    bool   
 7   capacity            270 non-null    int64  
dtypes: bool(1), float64(2), int64(1), object(4)
memory usage: 15.2+ KB


In [13]:
# df_pricing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   plan_id          2 non-null      object
 1   name             2 non-null      object
 2   currency         2 non-null      object
 3   price            2 non-null      int64 
 4   description      2 non-null      object
 5   is_taxable       2 non-null      bool  
 6   per_min_pricing  2 non-null      object
dtypes: bool(1), int64(1), object(5)
memory usage: 230.0+ bytes


,plan_id,name,currency,price,description,is_taxable,per_min_pricing
0,bike,Traditional bike charge,NOK,3,Cheapest possible fare with the system for a s...,False,"[{'start': 0, 'rate': 10, 'end': 0, 'interval'..."
1,ebike,Ebike charge,NOK,3,Cheapest possible fare with the system for a s...,False,"[{'start': 0, 'rate': 10, 'end': 0, 'interval'..."


# Link zum ERD
https://lucid.app/lucidchart/01f65829-ef67-49e9-9f5f-7e1fabab3e9b/edit?viewport_loc=-265%2C145%2C1975%2C1125%2C0_0&invitationId=inv_6ef8221a-3aaa-4b9b-a4d8-52262626b183

system_info, pricing und auto habe ich nicht beachtet. Diese Daten empfand ich als nicht wichtig, bzw. nicht hiilfreich. pricing hatte zwei default Werte. Auf der Webseite gibt es aber auch Abo-Pläne, die hier nicht beachtet wurden 